# Dask Assignment Excerise 6
Name: Revanth Chowdary Ganga

NetID: rg361

In [1]:
# Importing required libraries
import pandas as pd
from dask.distributed import Client
import dask.dataframe as dd

# Setting options
pd.set_option("mode.copy_on_write", True)

In [2]:
# path to raw file, file present in external SSD in my case
path = "/Volumes/Extreme SSD/arcos_all_washpost.tsv"

In [3]:
# Set required columns
cols_of_interest = ["BUYER_STATE", "BUYER_COUNTY", "TRANSACTION_DATE", "MME"]

# Setting these as we know the requirements from the project

## Dask Operations

In [4]:
# creating Dask Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59854,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:59866,Total threads: 2
Dashboard: http://127.0.0.1:59870/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:59857,


In [6]:
# Defining the Dask Dataframe
dask_df = dd.read_csv(
    path,
    sep="\t",
    usecols=cols_of_interest,
    dtype={
        "BUYER_STATE": "object",
        "BUYER_COUNTY": "object",
        "TRANSACTION_DATE": "object",
        "MME": "float64",
    },
    assume_missing=True,
)

In [7]:
# Processing Steps to be performed on the data

# generating year from the transaction date for grouping
dask_df["Year"] = dd.to_datetime(dask_df["TRANSACTION_DATE"], format="%Y-%m-%d").dt.year

# grouping by state, county and year and summing the MME
out_1 = (
    dask_df.groupby(["BUYER_STATE", "BUYER_COUNTY", "Year"])["MME"].sum().reset_index()
)

In [8]:
# Compute and move to pandas
df = out_1.compute()

## Regular Pandas Operations

In [9]:
# View Sample Output
df.sample(10)

,BUYER_STATE,BUYER_COUNTY,Year,MME
39324,IA,LYON,2006,9.163220e+05
18004,NE,KNOX,2015,1.435715e+06
42529,FL,BAKER,2011,3.278168e+07
29193,MI,HURON,2013,9.538195e+06
28733,IL,STEPHENSON,2006,5.696292e+06
34184,VA,CARROLL,2013,6.776718e+06
23951,MT,CASCADE,2019,1.608924e+07
5271,NC,HENDERSON,2018,3.314183e+07
33528,NY,CHEMUNG,2006,1.734762e+07
9795,IA,POWESHIEK,2019,2.084398e+06


In [10]:
# View Distribution of Numerical Columns
df.describe()

,Year,MME
count,43152.000000,4.315200e+04
mean,2012.499143,3.396096e+07
std,4.031802,1.891825e+08
min,2006.000000,1.816200e+01
25%,2009.000000,2.353492e+06
50%,2012.000000,8.144353e+06
75%,2016.000000,2.570302e+07
max,2019.000000,3.405486e+10


In [11]:
# check if any NAs are present
df.isna().sum()

BUYER_STATE     0
BUYER_COUNTY    0
Year            0
MME             0
dtype: int64

In [12]:
# View the Number of Rows and Data Types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43152 entries, 0 to 43151
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   BUYER_STATE   43152 non-null  string 
 1   BUYER_COUNTY  43152 non-null  string 
 2   Year          43152 non-null  int32  
 3   MME           43152 non-null  float64
dtypes: float64(1), int32(1), string(2)
memory usage: 1.2 MB


In [ ]:
# Data would get exported and saved as a parquet file in actual production as follows
# df.to_parquet("file_path.parquet")